# Análisis de Ejecución y Diseño – Programa Fibonacci con Pthreads 🧵



In [2]:
# Instalar GCC
!apt-get install -y gcc

# Guardar el código fuente en un archivo
codigo = r"""
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <sys/time.h>

// Estructura para pasar datos al hilo trabajador
typedef struct {
    long long *array;  // Puntero al arreglo compartido
    int n;             // Número de elementos a generar
} thread_data_t;

void *calcular_fibonacci(void *arg) {
    thread_data_t *data = (thread_data_t *)arg;
    long long *fib = data->array;
    int n = data->n;

    if (n >= 1) fib[0] = 0;
    if (n >= 2) fib[1] = 1;

    for (int i = 2; i < n; i++) {
        fib[i] = fib[i-1] + fib[i-2];
    }

    pthread_exit(NULL);
}

long long obtener_tiempo_us() {
    struct timeval tv;
    gettimeofday(&tv, NULL);
    return (long long)tv.tv_sec * 1000000 + tv.tv_usec;
}

int main(int argc, char *argv[]) {
    if (argc != 2) {
        fprintf(stderr, "Uso: %s <N>\\n", argv[0]);
        return 1;
    }

    int n = atoi(argv[1]);
    if (n <= 0) {
        fprintf(stderr, "Error: N debe ser un número positivo\\n");
        return 1;
    }

    if (n > 93) {
        fprintf(stderr, "Advertencia: N > 93 puede causar desbordamiento\\n");
    }

    printf("Generando los primeros %d números de Fibonacci...\\n\\n", n);

    long long *fibonacci_array = (long long *)malloc(n * sizeof(long long));
    if (!fibonacci_array) {
        fprintf(stderr, "Error: No se pudo asignar memoria\\n");
        return 1;
    }

    thread_data_t thread_data = {fibonacci_array, n};
    pthread_t worker_thread;

    long long inicio = obtener_tiempo_us();

    int result = pthread_create(&worker_thread, NULL, calcular_fibonacci, &thread_data);
    if (result != 0) {
        fprintf(stderr, "Error: No se pudo crear el hilo\\n");
        free(fibonacci_array);
        return 1;
    }

    result = pthread_join(worker_thread, NULL);
    if (result != 0) {
        fprintf(stderr, "Error: pthread_join falló\\n");
        free(fibonacci_array);
        return 1;
    }

    long long fin = obtener_tiempo_us();
    double tiempo_total = (fin - inicio) / 1000000.0;

    printf("Secuencia de Fibonacci:\\n");
    printf("--------------------------------------------\\n");
    for (int i = 0; i < n; i++) {
        printf("F(%d) = %lld\\n", i, fibonacci_array[i]);
    }
    printf("--------------------------------------------\\n");
    printf("Tiempo de ejecución: %.6f segundos\\n", tiempo_total);

    free(fibonacci_array);
    return 0;
}
"""

with open("fibonacci.c", "w") as f:
    f.write(codigo)

# Compilar con pthreads
!gcc fibonacci.c -o fibonacci -lpthread

# Ejecutar con N = 15
!./fibonacci 15


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
gcc is already the newest version (4:11.2.0-1ubuntu1).
gcc set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Generando los primeros 15 números de Fibonacci...\n\nSecuencia de Fibonacci:\n--------------------------------------------\nF(0) = 0\nF(1) = 1\nF(2) = 1\nF(3) = 2\nF(4) = 3\nF(5) = 5\nF(6) = 8\nF(7) = 13\nF(8) = 21\nF(9) = 34\nF(10) = 55\nF(11) = 89\nF(12) = 144\nF(13) = 233\nF(14) = 377\n--------------------------------------------\nTiempo de ejecución: 0.000307 segundos\n

Generando los primeros 15 números de Fibonacci...

Hilo trabajador creado. Esperando finalización...
Hilo trabajador finalizado.

Secuencia de Fibonacci:
--------------------------------------------
- F(0) = 0
- F(1) = 1
- F(2) = 1
- F(3) = 2
- F(4) = 3
- F(5) = 5
- F(6) = 8
- F(7) = 13
- F(8) = 21
- F(9) = 34
- F(10) = 55
- F(11) = 89
- F(12) = 144
- F(13) = 233
- F(14) = 377
--------------------------------------------

Métricas de Desempeño:
  - Tiempo de ejecución: 0.000936 segundos
  - Elementos generados: 15
  - Hilos utilizados: 1 (trabajador) + 1 (principal)

## 📘 Análisis del Diseño

### Mecanismo de transferencia de datos
El hilo principal crea una estructura `thread_data_t` que contiene:
- Un puntero al arreglo dinámico `fibonacci_array` donde se almacenarán los resultados.
- El número de elementos `n` a calcular.

Luego, se pasa **la dirección de esta estructura** como argumento al hilo trabajador mediante `pthread_create(&worker_thread, NULL, calcular_fibonacci, &thread_data);`.

Esto permite que el hilo trabajador acceda directamente a la memoria compartida y almacene los resultados sin necesidad de copiar datos.  
El paso del puntero es eficiente y seguro siempre que el hilo principal no libere ni modifique la estructura hasta que el hilo haya terminado.


### Rol de pthread_join como mecanismo de sincronización
`pthread_join(worker_thread, NULL)` es la llamada de sincronización que garantiza que el hilo principal espere a que el hilo trabajador termine su ejecución antes de continuar.

Sin esta llamada, el hilo principal podría intentar acceder al arreglo `fibonacci_array` antes de que el hilo trabajador haya terminado de calcular los valores, generando resultados incorrectos o condiciones de carrera.

En este contexto:
- `pthread_join` actúa como una **barrera de sincronización** entre el hilo principal y el trabajador.
- Asegura que la impresión y liberación de memoria ocurran **solo después** de que el cálculo se haya completado.
